<a href="https://colab.research.google.com/github/BlackBird-BB/BlackBird-BB.github.io/blob/master/meisai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import time

import PIL.Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from moudle import RNN, TrainSet

parameters = {
    "gold": {
        "file": "./data/LBMA-GOLD_repaired.csv",
        "train": {
            "logdir": "./logs/gold/train",
            "datadir": "./data/gold/train"
        },
        "pred": {
            "logdir": "./logs/gold/pred",
            "datadir": "./data/gold/pred"
        }
    },
    "bc": {
        "file": "./data/BCHAIN-MKPRU.csv",
        "train": {
            "logdir": "./logs/bc/train",
            "datadir": "./data/bc/train"
        },
        "pred": {
            "logdir": "./logs/bc/pred",
            "datadir": "./data/bc/pred"
        }
    }
}
table = parameters["bc"]
DELAY = 10
LR = 0.001
EPOCH = 1500
BATCH = 10
DAYS = 30
TRAIN_MODE = True

df = pd.read_csv(table['file'])


def splite(data, predict=False):
    y = data
    end = len(y)
    y = y[DELAY:]
    if not predict:
        end = end - 1
        y = y[:-1]
    x = []
    for i in range(DELAY, end):
        tmp = data[i - DELAY:i]
        tmp = torch.flatten(torch.tensor(tmp)).numpy()
        tmp = tmp.tolist()
        tmp.append(y[i - DELAY])
        x.append(tmp)
    return x


def cal_rsi(x):
    poi = []
    neg = []
    rsi = [0]
    for i in range(1, len(x)):
        per = (x[i] - x[i - 1]) / x[i - 1]
        if per > 0:
            poi.append(per)
        else:
            neg.append(per)
        rsi.append(100 * sum(poi) / (sum(poi) + (-1 * sum(neg))))
    return rsi


def draw(train_numpy, st, title, writer, rnn):
    y = []
    df_numpy = df.to_numpy()
    orig_data = [i[1] for i in df_numpy[:100]]
    data = pd.DataFrame(train_numpy)
    data = data[0].to_numpy()
    mean = np.mean(train_numpy)
    std = np.std(train_numpy)
    data = (data - mean) / std
    for i in range(DELAY, len(data)):
        x = data[i - DELAY:i]
        tmp = rnn(torch.tensor(x).cuda().float()).cpu()
        tmp = torch.flatten(tmp).detach().numpy()
        y.append(tmp * std + mean)

    fig = plt.figure()
    plt.plot([i+DELAY for i in range(len(y))], y, 'b')
    plt.plot([i for i in range(100)], orig_data, 'r')
    plt.savefig(f"{table['train']['datadir']}/tmp.png")
    plt.close()

    image = PIL.Image.open(f"{table['train']['datadir']}/tmp.png")
    image = np.array(image)
    writer.add_image(title, image, st, dataformats='HWC')

    torch.save(rnn.state_dict(), f"{table['train']['datadir']}/{title}_{st}.pth")


def regression(datas, title, writer, detail=True):
    datas = [i[1] for i in datas]

    train_numpy = splite(datas)
    mean = np.mean(train_numpy)
    std = np.std(train_numpy)
    df_numpy = (train_numpy - mean) / std

    trainset = TrainSet(torch.tensor(df_numpy))
    trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True)

    rnn = RNN(DELAY).cuda()
    loss_fnc = torch.nn.MSELoss().cuda()
    opti = torch.optim.Adam(rnn.parameters(), lr=LR)
    for step in range(EPOCH):
        loss = None
        for tx, ty in trainloader:
            tx = tx.cuda()
            ty = ty.cuda()
            output = rnn(torch.unsqueeze(tx, dim=0))
            loss = loss_fnc(torch.squeeze(output), ty)
            opti.zero_grad()
            loss.backward()
            opti.step()
        print(step, loss)
        if detail and (step + 1) % 50 == 0:
            draw(train_numpy, step, title, writer, rnn)
    return rnn


def prediction(datas, mdl):
    datas = [i[1] for i in datas]
    for day in range(DAYS):
        x = datas
        train_numpy = splite(x, predict=True)
        mean = np.mean(train_numpy)
        std = np.std(train_numpy)
        df_numpy = (train_numpy - mean) / std
        x = df_numpy[-1][:-1]
        predict_x = torch.tensor(x).cuda().float()
        output = mdl(predict_x).cpu() * std + mean
        datas.append(output.item())
    return datas


def prediction_eval(orig_data, prd_data, mdl, writer):
    orig_data = [i[1] for i in orig_data]
    cnt = (len(orig_data) - 100) / 30
    plt.figure()
    plt.plot([i for i in range(len(orig_data))], orig_data, "r")
    plt.plot([i for i in range(len(prd_data))], prd_data, "b")
    plt.legend()
    plt.savefig(f"{table['pred']['datadir']}/prediction_eval_{cnt}.png")
    plt.close()
    time.sleep(0.5)
    image = PIL.Image.open(f"{table['pred']['datadir']}/prediction_eval_{cnt}.png")
    image = np.array(image)
    writer.add_image("Prediction_eval", image, cnt, dataformats='HWC')

    torch.save(mdl.state_dict(), f"{table['pred']['datadir']}/mdl_{cnt}.pth")
    with open(f"{table['pred']['datadir']}/prd_data_{cnt}.txt", "w+") as f:
        f.write(f"The DataLength: {len(orig_data)}\n")
        f.write("And the last 30 Data were pridicted by the model.\n")
        f.write(f"Date:\n")
        for i in prd_data:
            f.write(f"{i} ")
    with open(f"{table['pred']['datadir']}/prd_data_all.txt", "a+") as f:
        do_pred = prd_data[-30:]
        for i in do_pred:
            f.write(f"{i} ")
    # writer.add_graph(mdl, torch.zeros(mdl.delays))


def call_regression():
    global DELAY, LR, BATCH, EPOCH
    print(f"NOW is running {LR}_{EPOCH}_{BATCH}")
    datas = df.to_numpy()[:100]
    writer = SummaryWriter(table['train']['logdir'])

    for DELAY in [10, 30]:
        for LR in [0.1, 0.01, 0.001, 0.0001]:
            for BATCH in [1, 10]:
                regression(datas, f"{DELAY}_{LR}_{BATCH}", writer)
                torch.cuda.empty_cache()

    writer.close()


def call_predict():
    writer = SummaryWriter(table['pred']['logdir'])
    df_numpy = df.to_numpy()
    for i in range(100, len(df_numpy), 30):
        datas = df_numpy[:i]
        mdl = regression(datas, "", writer, False)
        predict_result = prediction(datas, mdl)
        prediction_eval(df_numpy[:i + 30], predict_result, mdl, writer)
        torch.cuda.empty_cache()

    writer.close()


if __name__ == "__main__":
    # os.chdir(sys.path[0])
    if TRAIN_MODE:
        call_regression()
    else:
        call_predict()


NOW is running 0.001_1500_10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


流式输出内容被截断，只能显示最后 5000 行内容。
1000 tensor(2.3138e-06, device='cuda:0', grad_fn=<MseLossBackward0>)
1001 tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward0>)
1002 tensor(2.9136e-07, device='cuda:0', grad_fn=<MseLossBackward0>)
1003 tensor(0.0010, device='cuda:0', grad_fn=<MseLossBackward0>)
1004 tensor(4.3302e-06, device='cuda:0', grad_fn=<MseLossBackward0>)
1005 tensor(2.7919e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
1006 tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward0>)
1007 tensor(5.9181e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
1008 tensor(0.0002, device='cuda:0', grad_fn=<MseLossBackward0>)
1009 tensor(2.2800e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
1010 tensor(1.4997e-06, device='cuda:0', grad_fn=<MseLossBackward0>)
1011 tensor(0.0022, device='cuda:0', grad_fn=<MseLossBackward0>)
1012 tensor(0.0001, device='cuda:0', grad_fn=<MseLossBackward0>)
1013 tensor(3.1507e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
1014 tensor(2.9362e-06, device=